In [492]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

# vytvoříme si slovník s kraji a jejich labely, které jim web přiřadil
link = f'https://www.in-pocasi.cz/archiv/archiv.php?historie=2021-10-08&region=1'
raw = requests.get(link).content
soup = BeautifulSoup(raw, 'html.parser')
labels = []
names = []
for region in soup.find("select", {"name": "archiv_kraj"}).findAll("option"):
    labels.append(region["value"])

    if region.text == "Praha":
        name = "Hlavní město Praha"
    elif region.text == "Vysočina":
        name = "Kraj Vysočina"
    else: name = region.text + " kraj"
    names.append(name)
regions = dict(zip(labels, names))

# web obsahuje dva typy meteor. stanic a každý má trochu jiné měřené hodnoty, proto ze začátku vytvoříme 2 datasety
list_klimaticke_stanice = []
list_soukrome_stanice = []
date = "2021-10-08"
for region_label in list(regions.keys()): # projdeme všechny kraje

    link = f'https://www.in-pocasi.cz/archiv/archiv.php?historie={date}&region={region_label}'
    raw = requests.get(link).content
    soup = BeautifulSoup(raw, 'html.parser') # uložíme zdrojový kód
    soup_tables = soup.find('div',{'class':'typography'}).findAll("tbody") # tabulky s daty

    for type, table in enumerate(soup_tables):
        soup_rows = table.findAll("tr")

        for row in soup_rows:
            data = [] # list na data z konkrétní stanice
            
            for element in row.findAll("td"): # přiřazujeme všechny hodnoty
                data.append(element.text)
            data.append(regions[region_label]) # přiřadíme kraj
            data.append(date) #přiřadíme datum
            data.append(row.findAll("td")[0].find("a")["href"]) # uložíme si odkaz na meteor. stanici

            if type == 0: # přidáme do příslušného listu
                list_klimaticke_stanice.append(data)
            else: list_soukrome_stanice.append(data)

# vytvoříme datasety s neupravenými daty (pouze stringy)
cols_klima = ["Stanice", "Maximální teplota (°C)", "Minimální teplota (°C)", "Náraz větru (km/h)", "Srážky (mm)", "Sněhová pokrývka (cm)", "Sluneční svit (hod)", "Kraj", "Datum", "Odkaz"]
cols_soukr = ["Stanice", "Maximální teplota (°C)", "Minimální teplota (°C)", "Náraz větru (km/h)", "Srážky (mm)", "Nejvyšší tlak (hPa)", "Nejvyšší vlhkost (%)", "Kraj", "Datum", "Odkaz"]
raw_df_klima = pd.DataFrame(list_klimaticke_stanice, columns=cols_klima)
raw_df_soukr = pd.DataFrame(list_soukrome_stanice, columns=cols_soukr)

# převedeme data na čísla
def substitute(string):
    parts = string.split(" ")
    if len(parts) == 1:
        return np.nan
    elif len(parts) == 2:
        try:
            return float(parts[0])
        except ValueError as err:
            print("Inappropriate format: ", err) 
    else: raise ValueError("Inappropriate format!")

def modify_values(init_df):
    df = init_df.copy()
    for col in df.columns[1:7]:
        df[col] = df[col].apply(substitute)
    return df

df_klima = modify_values(raw_df_klima)
df_soukr = modify_values(raw_df_soukr)

# ciselnik stazen z https://www.czso.cz
ciselnik = pd.read_csv('data/ciselnik_uzemi_CR_1_1_2021.csv')
df = ciselnik.iloc[:, :2] # začátek datasetu

# data z https://www.rozpocetobce.cz
coords = pd.read_csv('data/coords.csv')

# nalezneme chybné řádky a smažeme:
# for i, mun in enumerate(coords['OBEC_CSU_KOD']):
#     if len(mun) != 6: 
#         print(f"row {i} has invalid length!")
coords = coords.drop(index= 5835)
# změníme datový typ
coords = coords.astype({'OBEC_CSU_KOD': int})
# přidáme data do datasetu
df = df.merge(coords, how="outer", left_on= "kod_obec", right_on= "OBEC_CSU_KOD").drop(["OBEC_NAZEV", "OBEC_CSU_KOD"], axis=1)

# vyscrapujeme potřebná data z wikipedie (musíme ručně poskytnout odkazy)
missing_id = list(df[df["LONGITUDE"].isna()]["kod_obec"])
missing_data = []
links = ['https://cs.wikipedia.org/wiki/Město_Libavá', 'https://cs.wikipedia.org/wiki/Bražec_(okres_Karlovy_Vary)', 
'https://cs.wikipedia.org/wiki/Vojenský_újezd_Boletice', 'https://cs.wikipedia.org/wiki/Vojenský_újezd_Březina',
'https://cs.wikipedia.org/wiki/Doupovské_Hradiště', 'https://cs.wikipedia.org/wiki/Hlučín',
'https://cs.wikipedia.org/wiki/Vojenský_újezd_Hradiště', 'https://cs.wikipedia.org/wiki/Kozlov_(okres_Olomouc)',
'https://cs.wikipedia.org/wiki/Krhová', 'https://cs.wikipedia.org/wiki/Vojenský_újezd_Libavá',
'https://cs.wikipedia.org/wiki/Libhošť', 'https://cs.wikipedia.org/wiki/Luboměř_pod_Strážnou',
'https://cs.wikipedia.org/wiki/Město_Libavá', 'https://cs.wikipedia.org/wiki/Petrov_nad_Desnou',
'https://cs.wikipedia.org/wiki/Poličná', 'https://cs.wikipedia.org/wiki/Polná_na_Šumavě',
'https://cs.wikipedia.org/wiki/Želešice']
for l in links:
    source = requests.get(l).content
    soup_obce = BeautifulSoup(source, 'html.parser')
    lat_string = soup_obce.find('span', {'class': 'coordinates'}).find_all('span')[0].text
    lon_string = soup_obce.find('span', {'class': 'coordinates'}).find_all('span')[1].text
    
    try: 
        lat = float(lat_string[:lat_string.find('°')]) + float(lat_string[lat_string.find('°') + 1:lat_string.find('′')]) / 60 + float(lat_string[lat_string.find('′') + 1: lat_string.find('″')]) / 3600
    except:
         lat = float(lat_string[:lat_string.find('°')]) + float(lat_string[lat_string.find('°') + 1:lat_string.find('′')]) / 60
    
    try: 
        lon = float(lon_string[:lon_string.find('°')]) + float(lon_string[lon_string.find('°') + 1:lon_string.find('′')]) / 60 + float(lon_string[lon_string.find('′') + 1: lon_string.find('″')]) / 3600
    except:
        lon = float(lon_string[:lon_string.find('°')]) + float(lon_string[lon_string.find('°') + 1:lon_string.find('′')]) / 60 
    
    try:
        psc = ''.join((soup_obce.find('th', string = "PSČ").find_next('td').text).split(' '))
    except: psc = np.nan
    adresa = str(soup_obce.find('th', string = "Kontakt").find_next('td')).split('<br/>')[0][4:]
    ico = np.nan
    try:
        email = soup_obce.find('th', string = "Kontakt").find_next('td').find('a')['href'][7:]
    except: 
        email = np.nan
    missing_data.append([adresa, psc, ico, l, lat, lon, email])

# doplníme chybějící hodnoty
for i,id in enumerate(missing_id):
    for j in range(7):
        df.loc[df["kod_obec"] == id, df.columns[3 + j]] = missing_data[i][j]

# funkce na přiřazení kódů obcí k meteostanicím
def get_mun_codes(mun_data):    
    data = mun_data.copy()
    mun_codes = []
    combinations = pd.Series(list(ciselnik["nazev_obec"] + ciselnik["nazev_kraj"]))
    for i in range(data.shape[0]):
        if sum(data.iloc[i, 0] + data.iloc[i, 7] == combinations) == 1:
            mun_codes.append(ciselnik["kod_obec"][data.iloc[i, 0] + data.iloc[i, 7] == combinations].values[0])
        elif "-" in data.iloc[i, 0]:
            try:
                mun_codes.append(ciselnik["kod_obec"][data.iloc[i, 0][:data.iloc[i, 0].find("-")-1] + data.iloc[i, 7] == combinations].values[0])
            except: 
                mun_codes.append(np.nan)
        else:
            mun_codes.append(np.nan)
    data["kod_obec"] = mun_codes
    return data

df_klima = get_mun_codes(df_klima) # přiřadíme kódy
# dataset obsahuje chybějící hodnoty, přepíšeme ručně názvy stanic (na nejbližší obci) a poté znovu aplikujeme funkci get_mun_codes()
new_mun = dict(zip(df_klima[df_klima["kod_obec"].isna()]["Stanice"],["Stachy", "Pec pod Sněžkou", "Pec pod Sněžkou", "Budišov nad Budišovkou", "Ostravice", 
"Ostružná", "Velemín", "Kadaň", "Křešín", "Kunovice"]))
for stanice in list(new_mun.keys()):
    df_klima.loc[df_klima["Stanice"] == stanice, "Stanice"] = new_mun[stanice]
df_klima = get_mun_codes(df_klima)
df_klima.loc[df_klima["Stanice"] == "Kunovice", "kod_obec"] = 550744 # 1 hodnotu musíme doplnit ručně

In [6]:
from geopy.distance import great_circle
import gzip
import pickle

7.319262734964349


In [220]:
# spočítáme všechny vzájemné vzdálenosti mezi obcemi, vytvoříme data frame a uložíme

# length = df.shape[0]
# distances_np = np.zeros((length, length))
# for i in range(length):
#     for j in range(length):
#         distances_np[i, j] = great_circle((df["LATITUDE"][i], df["LONGITUDE"][i]), (df["LATITUDE"][j], df["LONGITUDE"][j])).km

# dist_df = pd.DataFrame(distances_np, index=df["kod_obec"], columns=df["kod_obec"])
# with gzip.open("data/municipal_distances_df", "wb") as dists:
#     pickle.dump(dist_df, dists)

In [246]:
# kód na načtení matice vzdáleností
with gzip.open('data/municipal_distances_df', 'rb') as f:
    distances = pickle.load(f)


In [497]:
ciselnik[ciselnik["nazev_obec"]=="Pohorská Ves"]

,kod_obec,nazev_obec,status,kod_obec_povereny_ob_urad,nazev_obec_povereny_ob_urad,kod_orp,nazev_orp,kod_okres,nazev_okres,kod_kraj,nazev_kraj,kod_region,nazev_region
3925,545694,Pohorská Ves,O,31061,Kaplice,3106,Kaplice,CZ0312,Český Krumlov,CZ031,Jihočeský kraj,CZ03,Jihozápad


In [498]:
distances

kod_obec,554979,531367,535826,581291,547786,547981,598925,576077,549258,563528,...,569801,574091,534668,573850,586013,555762,541575,572641,562424,564338
kod_obec,,,,,,,,,,,,,,,,,,,,,
554979,0.000000,193.256721,196.153377,299.902577,234.383647,276.995813,412.198003,232.166607,163.497130,178.992991,...,224.902446,202.921833,197.647530,182.101169,359.660847,39.353840,303.811337,223.547757,249.100163,128.761312
531367,193.256721,0.000000,114.193717,109.349602,98.149240,88.862256,223.628880,56.372955,104.330297,101.017889,...,36.126466,11.954867,6.710485,49.441799,167.090104,162.701483,130.599668,32.225311,100.400818,81.435666
535826,196.153377,114.193717,0.000000,157.680594,212.241928,183.760364,301.306672,167.740830,32.957702,202.875974,...,110.074238,123.919651,120.333665,156.923392,215.959524,156.809102,234.753674,136.109739,70.709902,80.555383
581291,299.902577,109.349602,157.680594,0.000000,151.759850,69.731465,145.060445,102.634996,170.967839,190.194832,...,75.127170,102.607530,106.778503,144.212547,61.406546,266.346212,116.539326,86.041023,92.135657,176.309468
547786,234.383647,98.149240,212.241928,151.759850,0.000000,85.774698,195.250807,53.340262,199.372006,60.804377,...,115.206422,89.442510,92.278761,62.323934,186.228890,217.011980,78.069090,84.665497,189.748841,163.939271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555762,39.353840,162.701483,156.809102,266.346212,217.011980,249.452875,385.110663,206.932028,124.194704,167.189571,...,192.009757,173.248469,167.759189,159.665786,326.928237,0.000000,281.138505,194.181982,210.996581,91.876601
541575,303.811337,130.599668,234.753674,116.539326,78.069090,53.386205,117.232313,75.959564,232.546143,138.331448,...,124.698540,118.681098,123.967576,121.921164,126.546604,281.138505,0.000000,100.876138,189.102794,210.962185
572641,223.547757,32.225311,136.109739,86.041023,84.665497,56.669021,191.508489,33.004033,131.701562,107.038283,...,30.617080,20.955485,26.512314,58.439365,139.655241,194.181982,100.876138,0.000000,105.114169,113.438297
